# Clustering
#### Part of the course on "Foundations of machine learning", Department of Mathematics and Statistics, University of Turku, Finland
#### Lectures available on YouTube: https://youtube.com/playlist?list=PLbkSohdmxoVAZ9DEHEWHjeGK7Ei-DjKHI&si=Msu74_I0qhLrRWcu
#### Code available on GitHub: https://github.com/ionpetre/FoundML_course_assignments

#### This notebook is based on the following sources: 
> https://www.kaggle.com/code/mcmuralishclint/decision-tree-regression (MURALISH)

> https://www.kaggle.com/code/marklvl/decision-tree-regressor-on-bike-sharing-dataset (MARK KAGHAZGARIAN)

> https://www.kaggle.com/code/akashchola/decision-tree-for-classification-regression (AKASH GUPTA)

> https://www.kaggle.com/code/arunmohan003/pruning-decision-trees-tutorial (ARUNMOHAN_003)

Decision trees are a popular machine learning technique that offers a structured and intuitive way to make decisions based on data. They can be seen as a flowchart-like structure where each node represents a decision (a test on a feature), and each branch represents an outcome of that test. Decision trees are widely used for classification and regression tasks, as they excel in breaking down complex decision-making processes into a series of simple choices. One of the key advantages of decision trees is their interpretability, making them an excellent choice when transparency and understanding of the underlying logic are important. Additionally, decision trees can handle both categorical and numerical data, making them versatile tools in the data scientist's toolkit. However, they are prone to overfitting, especially when they become deep and complex, so careful pruning and tuning are often required to ensure optimal performance.

In this noteebook we demonstrate how decisoin trees can be used for classification and for regrerssion. We use the UCI heart disease dataset for the classifier and a dataset on 2005 used GM car prices for the rergressor. We discuss the training of decision trees and how to prune them in different ways to address their tendency to overfit. 

#### Load the libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import tree
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score as R2

In [ ]:
# Reset the seed of the random number generator, for reproducibility purposes

import os

def reset_seed(SEED = 0):
    """Reset the seed for every random library in use (System, numpy)"""

    os.environ['PYTHONHASHSEED']=str(SEED)
    np.random.seed(SEED)


reset_seed(220)

# Prepare the style of the plots

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

## I. Demo decision trees on the UCI heart disease dataset

#### The UCI heart disease dataset: https://archive.ics.uci.edu/dataset/45/heart+disease

The UCI Heart Disease dataset is a well-known dataset used in machine learning and data analysis to study and predict the presence of heart disease in individuals. It is often referred to as the "Cleveland Heart Disease dataset" because it was collected from the Cleveland Clinic Foundation in the late 1980s.

Here are some key details about the UCI Heart Disease dataset:

1. Data Source: The dataset was collected from the Cleveland Clinic Foundation's Heart Disease Institute. The original dataset had several contributors, including Robert Detrano, Don Brownlee, and Wesley Turner.

2. Data Description: The UCI Heart Disease dataset consists of 303 instances, each representing a patient, and contains 76 attributes. However, only 14 of these attributes are typically used in analysis and modeling. These attributes include features such as age, sex, chest pain type, resting blood pressure, cholesterol level, maximum heart rate, exercise-induced angina, and more.

3. Target Variable: The primary target variable in this dataset is the presence of heart disease, where '0' typically indicates no heart disease and '1' indicates the presence of heart disease. This binary classification task makes it a popular choice for predictive modeling.

4. Purpose: The UCI Heart Disease dataset is commonly used for research, practice, and educational purposes in the field of cardiovascular medicine, as well as in machine learning and data science. Researchers and data scientists use this dataset to develop predictive models for diagnosing heart disease and assessing cardiovascular risk factors.

5. Data Availability: The UCI Heart Disease dataset is publicly available and can be accessed through the UCI Machine Learning Repository or various data science platforms and libraries.

In [ ]:
from sklearn.datasets import fetch_openml

X, _ = fetch_openml(
    data_id=43823,
    as_frame=True,
    return_X_y=True,
    parser = 'auto'
)


In [ ]:
X.info()

In [ ]:
X

In [ ]:
# The target feature is 'Heart_Disease'. We save it in the variable y and encode it as 0/1.

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(X['Heart_Disease']))
y.value_counts()

In [ ]:
# We drop the target feature 'Heart_Disease' from the X dataset. 

X = X.drop('Heart_Disease', axis=1)
X

In [ ]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=150, 
    stratify=y,
    shuffle=True
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    test_size=0.25, 
    random_state=150, 
    stratify=y_train_valid,
    shuffle=True
)

# convert to pandas dataframe
X_train = pd.DataFrame(X_train, columns=X.columns)
X_valid = pd.DataFrame(X_valid, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)
y_train = pd.DataFrame(y_train, columns=y.columns)
y_valid = pd.DataFrame(y_valid, columns=y.columns)
y_test = pd.DataFrame(y_test, columns=y.columns)

del X
del y

In [ ]:
print(X_train.info(), "\n", X_valid.info(), "\n", X_test.info())
print(y_train.value_counts(), "\n", y_valid.value_counts(), "\n", y_test.value_counts())

#### Train a decision tree classifier with its default setup. 

In [ ]:
clf = tree.DecisionTreeClassifier(random_state=2023)
clf.fit(X_train,y_train)

#### Display the decision tree. 
Each node is shown with its internal characteristics: the decision rule, the impurity, the class balance. The decision that could be made in each node based on the class balance is indicated through the coloring of the node. The more orange the node, the clearer the decision for "Not heart disease". The more blue the node, the clearer the decision for "Heart disease".

In [ ]:
plt.figure(figsize=(60,30))
features = list(X_train.columns.values)
classes = ['NO','YES']
tree.plot_tree(
    clf,
    feature_names = features,
    class_names = classes,
    filled = True,
    proportion = False,
    impurity = True, 
    rounded = True,
    fontsize = 16
)
plt.show()

#### Check some of the characteristics of the tree

In [ ]:
print("Depth of the decision tree: ", clf.get_depth())
print("Number of leaves: ", clf.get_n_leaves())

#### Check the quality of the predictions through the accuracy score (on train and validation) and through the confusion matrix.

In [ ]:
def plot_confusionmatrix(y_train_pred,y_train,dom):
    print(f'{dom} confusion matrix:')
    cf = confusion_matrix(y_train_pred,y_train, normalize=None)
    plt.figure(figsize=(4,3))
    sns.heatmap(cf,annot=True,yticklabels=classes
               ,xticklabels=classes,cmap='Blues', fmt='g')
    plt.tight_layout()
    plt.show()

In [ ]:
y_train_pred = clf.predict(X_train)
y_valid_pred = clf.predict(X_valid)

print(f'Train score {accuracy_score(y_train, y_train_pred)}')
print(f'Validation score {accuracy_score(y_valid, y_valid_pred)}')

plot_confusionmatrix(y_train, y_train_pred, dom='Train')
plot_confusionmatrix(y_valid, y_valid_pred, dom='Validation')

#### The confusion matrix shows that the model gets perfect predictions on the training set, and only about 80% correct on the validation set: 23/30 for class 'NO' and 16/24 for class 'YES'. This is a sign that the classifier is overfit. This is a typical problem for decision trees. 

We explore next some pruning techniques: limiting the depth, the number of leaves, and the minimal-cost complexity pruning. We also check if using a different split criterion may help. 

Minimal cost complexity pruning (also known as minimal cost complexity pruning, cost-complexity pruning, or simply cost pruning) is a method used to prune decision trees to avoid overfitting. Decision trees, when fully grown, can become overly complex and capture noise in the training data, leading to poor generalization on unseen data. Pruning helps simplify the tree by removing branches and nodes that do not significantly contribute to its predictive power.

The main idea behind minimal cost complexity pruning is to add a penalty term for tree complexity during the pruning process. The penalty term is controlled by a tuning parameter, typically denoted as "alpha" (α). By varying the value of alpha, you can adjust the trade-off between tree complexity and predictive accuracy.

Here's how minimal cost complexity pruning works:

> Initial Tree Construction: Start by growing a full, potentially overfit decision tree using a standard tree-building algorithm.

> Calculate Cost Complexity Measure: For each node in the tree, calculate a cost complexity measure that combines two factors: the impurity of the node (such as Gini impurity or entropy) and the size of the subtree rooted at that node.

> Introduce the Complexity Penalty: Introduce the tuning parameter alpha (α), which controls the strength of the penalty. A small alpha value encourages a more complex tree, while a large alpha value encourages a simpler tree.

> Pruning Process: Starting from the leaves and moving toward the root of the tree, evaluate whether the "cost" of removing a subtree (as determined by the complexity measure) is smaller than the "benefit" in terms of reduced complexity and potential improvements in predictive accuracy. Prune subtrees that do not pass this cost-benefit analysis.

> Select the Optimal Alpha: You can use techniques like cross-validation to find the optimal value of alpha that balances model complexity and predictive accuracy.

> Final Pruned Tree: The result is a pruned decision tree with a balanced trade-off between model complexity and predictive performance. The pruned tree is typically more interpretable and less prone to overfitting.

Minimal cost complexity pruning helps ensure that the final decision tree is a good compromise between capturing the training data's patterns and avoiding overfitting. 

We apply a GridSearch to explore various combinations of parameter values. We do this with a 5-fold cross-validation apporach. 

In [ ]:
param_grid = {
              "min_samples_split": [2, 3, 5, 7],
              "max_depth": [3, 4, 5, 6, 7, 8, 9],
              "max_leaf_nodes": [10, 20, 30, 50],
              'ccp_alpha': [0, 0.1, 0.2]
              }


clf = tree.DecisionTreeClassifier(random_state=2023)
gcv = GridSearchCV(
    estimator = clf,
    cv = 5,
    param_grid = param_grid,
    refit = True,
    n_jobs = -1,
    verbose = 1,
)
gcv.fit(X_train_valid,y_train_valid)

best_clf = gcv.best_estimator_
best_clf.fit(X_train,y_train)
y_train_pred = best_clf.predict(X_train)
y_valid_pred = best_clf.predict(X_valid)

print(f'Train score {accuracy_score(y_train, y_train_pred)}')
print(f'Validation score {accuracy_score(y_valid, y_valid_pred)}')
plot_confusionmatrix(y_train, y_train_pred, dom='Train')
plot_confusionmatrix(y_valid, y_valid_pred, dom='Validation')

Our hyper-parameter tuning worked out: we got a model that does better on the validation set, while remaining very good on the training set. Comparing the confusion matrices, we see that the model improved: 27/30 correct predictions in class 'NO' and 17/24 correct predictions in class 'YES'. We check its properties. 

In [ ]:
print("Depth of the improved decision tree: ", best_clf.get_depth())
print("Number of leaves: ", best_clf.get_n_leaves())

plt.figure(figsize=(60,30))
features = list(X_train.columns.values)
classes = ['NO','YES']
tree.plot_tree(
    best_clf,
    feature_names = features,
    class_names = classes,
    filled = True,
    proportion = False,
    impurity = True, 
    rounded = True,
    fontsize = 16
)
plt.show()

In [ ]:
# Check the parameters of the best model we got through GridSearchCV

best_clf.get_params()

In [ ]:
# Model's performance on the test dataset

y_test_pred = best_clf.predict(X_test)

print(f'Test score {accuracy_score(y_test, y_test_pred)}')
plot_confusionmatrix(y_test, y_test_pred, dom='Test')

In [ ]:
del X_train_valid
del y_train_valid
del X_train
del y_train
del X_valid
del y_valid
del X_test
del y_test

## Challenge: 

We will train a decision tree regressor on a dataset of used car prices. This is taken from the 2005 version of the Kelly Blue Book and concerns several models of General Motors cars. 

Link to the dataset: https://www.openml.org/search?type=data&status=any&id=44994&sort=runs

**Data Description**

Data frame of the suggested retail prices (column Price) and various characteristics of each car.

For this data set, a representative sample of over eight hundred 2005 GM cars were selected, then retail prices were calculated from the tables provided in the 2005 Central Edition of the Kelly Blue Book.

Attribute Description

All features describe different self-explanatory characteristics for the cars.

Price - target feature
Mileage
Cylinder
Doors
Cruise
Sound
Leather
Buick
Cadillac
Chevy
Pontiac
Saab
Saturn
convertible
coupe
hatchback
sedan
wagon

In [ ]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml(
    data_id=???,
    as_frame=True,
    return_X_y=True,
    parser = 'auto'
)

#### Q1. How many features you have in the dataset (not counting the target feature 'Price')?
#### Q2. Do you have missing values in your dataset? 

In [ ]:
# Split the data into train/validation/test. Use random_state=150 and the same ratios train/valid/test as in the demo part.
# Your code here


#### Q3. Train a decision tree regressor with the sklearn libary, where all parameters are left to their default values, except for random_state, to be set to 2023. What is the depth of the tree?
#### Q4. What is the number of its leaves? 
#### Q5. What is the mean squared error on the training dataset? 
#### Q6. What is the mean squared error on the validation dataset (0 decimals)?
#### Q7. What is the r2 score on the training dataset (2 decimals)? 
#### Q8. What is the r2 score on the validation dataset (2 decimals)? 
#### Q9. Do you consider this tree to be overfit? 

In [ ]:
# Train a decision tree regressor
# Your code here



#### Q10. Do a parameter grid search on the following combination of parameter values. What is the r2 score of the best model on the training data (2 decimals)?

param_grid = {
              "max_depth": [5, 10, 15, 20, 25],
              "max_leaf_nodes": [50, 100, 200, 400],
              'ccp_alpha': [0, 0.2, 0.4]
              }
              
#### Q11. What is the r2 score of the best model on the validation data (2 decimals)? 
#### Q12. Do you consider this model to be better than the one with the default parameter values? 
#### Q13. What is the depth of this model? 
#### Q14. What is the r2 score of the best model (either the initial one or the one obtained through the grid search) on the test data (2 decimals)? 

In [ ]:
# Your code here

